In [1]:
import json
from pathlib import Path

import pandas as pd

In [3]:
# load week 1 json
with open('data/09_13_2020.json', 'r') as f:
    data = json.load(f)

In [4]:
# step 1: limit to NFL slates
# there are NBA, MLB, etc. in this file
nfl = [item for item in data if 'QB' in item['startingPositions']]

In [5]:
# step 2: find the main slate
# most accurate way is to count teams from schedule
# without that data available, we want the 2nd most games
# here, we see that index 3 is main slate
[item['gameCount'] for item in nfl]

[3, 2, 3, 12, 15, 9]

In [6]:
ms = nfl[3]

In [7]:
# one thing of value to get are the slate players
df = pd.DataFrame(ms['slatePlayers'])

In [8]:
df.head()

,name,dkName,siteSlatePlayerId,slatePosition,salary,team,projectedOwnership,ecr,rgPlayerId,projectedFpts,statId,scheduleId,opponent,teamId,_id
0,Christian McCaffrey,Christian McCaffrey,15033483,RB/FLEX,10000,CAR,0.204,"{'average': 2.13, 'rank': 1}",36994,26.11,844666.0,70353,LVR,22.0,5f6108d6b582b575983696ea
1,Michael Thomas,Michael Thomas,15033799,WR/FLEX,9000,NOS,0.141,"{'average': 3.25, 'rank': 2}",35596,23.85,844838.0,70360,TBB,23.0,5f6108d6b582b575983696e9
2,Lamar Jackson,Lamar Jackson,15033397,QB,8100,BAL,0.139,"{'average': 1.5, 'rank': 1}",328097,26.22,844594.0,70350,CLE,1.0,5f6108d6b582b575983696e8
3,Dalvin Cook,Dalvin Cook,15033485,RB/FLEX,7900,MIN,0.092,"{'average': 5.63, 'rank': 4}",36885,20.51,844800.0,70358,GBP,8.0,5f6108d6b582b575983696e7
4,Julio Jones,Julio Jones,15033801,WR/FLEX,7700,ATL,0.121,"{'average': 9.25, 'rank': 6}",12896,20.51,844739.0,70356,SEA,21.0,5f6108d6b582b575983696e6


In [28]:
# fixes to make to data
# break out ecr
ecr_posavg = []
ecr_posrank = []
for row in df.itertuples():
    ecr_posavg.append(row.ecr['average'] if isinstance(row.ecr, dict) else None)
    ecr_posrank.append(row.ecr['rank'] if isinstance(row.ecr, dict) else None) 

In [29]:
df['ecr_posavg'] = ecr_posavg
df['ecr_posrank'] = ecr_posrank

In [31]:
# change slatePosition to pos
df['pos'] = df.slatePosition.str.split('/').str[0]

In [36]:
# select columns 
wanted = ['name', 'pos', 'team', 'opponent', 'salary', 'projectedOwnership', 'ecr_posavg', 'ecr_posrank', 'projectedFpts']
df.loc[:, wanted]

,name,pos,team,opponent,salary,projectedOwnership,ecr_posavg,ecr_posrank,projectedFpts
0,Christian McCaffrey,RB,CAR,LVR,10000,0.204,2.13,1.0,26.11
1,Michael Thomas,WR,NOS,TBB,9000,0.141,3.25,2.0,23.85
2,Lamar Jackson,QB,BAL,CLE,8100,0.139,1.50,1.0,26.22
3,Dalvin Cook,RB,MIN,GBP,7900,0.092,5.63,4.0,20.51
4,Julio Jones,WR,ATL,SEA,7700,0.121,9.25,6.0,20.51
...,...,...,...,...,...,...,...,...,...
667,Browns,DST,CLE,BAL,2200,NaN,NaN,NaN,NaN
668,Jets,DST,NYJ,BUF,2100,NaN,NaN,NaN,NaN
669,Cardinals,DST,ARI,SFO,2100,NaN,NaN,NaN,NaN
670,Dolphins,DST,MIA,NEP,2000,NaN,NaN,NaN,NaN
